In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.model_selection import train_test_split

In [147]:
data_train = pd.read_csv("train.csv").fillna(0)
y_train = np.array(pd.read_csv("train.csv")["Transported"].fillna(0).astype(float))
# data_train = data_train[["HomePlanet", "CryoSleep", "Cabin", "Destination", "Age", "VIP"]]
data_train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,28.234326,220.009318,448.434027,169.572300,304.588865,298.261820
std,14.912090,660.519050,1595.790627,598.007164,1125.562559,1134.126417
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,41.000000,61.000000,22.000000,53.000000,40.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [148]:

def transform(data):
    # PassengerId*, HomePlanet(Earth,Europa, Mars), CryoSleep*, Cabin(P or S)*,
    # Destination(TRAPPIST-1e,55 Cancri e, PSO J318.5-22), Age*, VIP*
    data.Age = data.Age / 79
    data.RoomService /= 14327
    data.FoodCourt /= 29813
    data.ShoppingMall /= 23492
    data.Spa /= 22408
    data.VRDeck /= 24133

    data.CryoSleep = (data.CryoSleep.astype(float) == 1.0).astype(int)
    data.VIP = (data.VIP.astype(float) == 1.0).astype(int)
    data.Cabin = [int("P" in i) for i in data.Cabin.astype(str)]

    home_planet_cat = pd.get_dummies(data.HomePlanet).astype(int)[["Earth", "Europa", "Mars"]]
    destination_cat = pd.get_dummies(data.Destination).astype(int)[["TRAPPIST-1e", "55 Cancri e", "PSO J318.5-22"]]
    return pd.concat([data, home_planet_cat, destination_cat], axis=1, ignore_index=False)


In [149]:
data = transform(data_train.copy())[
    ["CryoSleep", "Cabin", "Age", "VIP", "Earth", "Europa", "Mars", "TRAPPIST-1e", "55 Cancri e", "PSO J318.5-22","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
]
x_train = np.array(data.values)
x_train

array([[0.00000000e+00, 1.00000000e+00, 4.93670886e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.03797468e-01, ...,
        1.06419207e-03, 2.45001785e-02, 1.82322960e-03],
       [0.00000000e+00, 0.00000000e+00, 7.34177215e-01, ...,
        0.00000000e+00, 2.99669761e-01, 2.03041478e-03],
       ...,
       [0.00000000e+00, 0.00000000e+00, 3.29113924e-01, ...,
        7.96867019e-02, 4.46269190e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.05063291e-01, ...,
        0.00000000e+00, 1.57533024e-02, 1.34048813e-01],
       [0.00000000e+00, 0.00000000e+00, 5.56962025e-01, ...,
        0.00000000e+00, 0.00000000e+00, 4.97244437e-04]])

In [ ]:
model = tf.keras.Sequential([
    Input((15,)),
    Dense(64, "relu"),
    Dropout(0.2),
    Dense(16, "relu"),
    Dropout(0.2),
    Dense(1, "sigmoid")
])
model.compile("adam", tf.losses.binary_crossentropy, ["accuracy"])
his = model.fit(x_train, y_train, 64, epochs=50, validation_split=0.2).history
model.evaluate(x_train, y_train)

Epoch 1/50
109/109 [==============================] - 1s 1ms/step - loss: 0.6194 - accuracy: 0.6684 - val_loss: 0.5409 - val_accuracy: 0.7332
Epoch 2/50
109/109 [==============================] - 0s 647us/step - loss: 0.5545 - accuracy: 0.7222 - val_loss: 0.5171 - val_accuracy: 0.7470
Epoch 3/50
109/109 [==============================] - 0s 679us/step - loss: 0.5343 - accuracy: 0.7337 - val_loss: 0.4950 - val_accuracy: 0.7550
Epoch 4/50
109/109 [==============================] - 0s 655us/step - loss: 0.5136 - accuracy: 0.7432 - val_loss: 0.4837 - val_accuracy: 0.7648
Epoch 5/50
109/109 [==============================] - 0s 661us/step - loss: 0.5026 - accuracy: 0.7476 - val_loss: 0.4859 - val_accuracy: 0.7706
Epoch 6/50
109/109 [==============================] - 0s 805us/step - loss: 0.4916 - accuracy: 0.7574 - val_loss: 0.4658 - val_accuracy: 0.7780
Epoch 7/50
109/109 [==============================] - 0s 645us/step - loss: 0.4809 - accuracy: 0.7603 - val_loss: 0.4643 - val_accuracy: 0

In [ ]:
plt.plot(his["accuracy"], c="r")
plt.plot(his["val_accuracy"], c="b")
plt.show()

In [152]:
clf = RandomForestClassifier(n_estimators=400, max_depth=45)
x_tr, x_test, y_tr, y_test = train_test_split(x_train, y_train, test_size=0.2, shuffle=False)
print(x_tr.shape, y_tr.shape,x_test.shape,y_test.shape)

clf.fit(x_tr,y_tr)
clf.score(x_test,y_test)

(6954, 15) (6954,) (1739, 15) (1739,)


0.7866589994249569

In [153]:
clf = AdaBoostClassifier(n_estimators=800)
clf.fit(x_tr,y_tr)
clf.score(x_test,y_test)

0.7918343875790684

In [154]:
test = pd.read_csv("test.csv").fillna(0)
test = transform(test.copy())[
    ["CryoSleep", "Cabin", "Age", "VIP", "Earth", "Europa", "Mars", "TRAPPIST-1e", "55 Cancri e", "PSO J318.5-22","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
]
x_test = np.array(data.values)

In [155]:
sub = pd.read_csv("sample_submission").fillna(0)
sub.Transported = pd.DataFrame((model.predict(x_test) >= 0.5).astype(int)).astype(bool)
pd.DataFrame(sub[["PassengerId","Transported"]]).to_csv("resultNN.csv",index=False)

272/272 [==============================] - 0s 295us/step
